In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import configparser
import logging
import os
import sys

import numpy as np
import torch
import wandb
from architectures import TempRedUNet
from custom_losses import FocalLoss, LovaszSoftmax3d, SumFocalLovasz
from datasets import SparkDataset
from new_unet import UNet
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from training_inference_tools import (
    compute_class_weights,
    random_flip,
    random_flip_noise,
    sampler,
    test_function,
    training_step,
    weights_init,
    myTrainingManager
)

import unet

logger = logging.getLogger(__name__)

In [34]:
############################# fixed parameters #############################

# General params
logfile = None  # change this when publishing finished project on github
wandb_project_name = "TEST"
output_relative_path = "runs"  # directory where output, saved params and
# testing results are saved

# Dataset parameters
ignore_index = 4  # label ignored during training
num_classes = 4  # i.e., BG, sparks, waves, puffs
ndims = 3  # using 3D data

In [35]:
############################# configure logger #############################

# set verbosity
debug_mode = True#c.getboolean("general", "debug_mode", fallback=False)
verbosity = 3 if debug_mode else 2

level_map = {
    3: logging.DEBUG,
    2: logging.INFO,
    1: logging.WARNING,
    0: logging.ERROR,
}
log_level = level_map[verbosity]
log_handlers = (logging.StreamHandler(sys.stdout),)

logging.basicConfig(
    level=log_level,
    format="[{asctime}] [{levelname:^8s}] [{name:^12s}] <{lineno:^4d}> -- {message:s}",
    style="{",
    datefmt="%H:%M:%S",
    handlers=log_handlers,
)

In [36]:
############################# load config file #############################

CONFIG_FILE = os.path.join("config_files", "config.ini") # provare anche con nuovo training per vedere se ci mette tanto
#CONFIG_FILE = os.path.join("config_files", "config_focal_loss_updated_physio.ini")
c = configparser.ConfigParser()
if os.path.isfile(CONFIG_FILE):
    logger.info(f"Loading {CONFIG_FILE}")
    c.read(CONFIG_FILE)
else:
    logger.warning(
        f"No config file found at {CONFIG_FILE}, trying to use fallback values."
    )

[10:41:33] [  INFO  ] [  __main__  ] < 7  > -- Loading config_files\config.ini


In [37]:
############################## set parameters ##############################

params = {}

# training params
params["run_name"] = c.get("training", "run_name", fallback="TEST")  # Run name
params["load_run_name"] = c.get("training", "load_run_name", fallback=None)
params["load_epoch"] = c.getint("training", "load_epoch", fallback=0)
params["train_epochs"] = c.getint("training", "train_epochs", fallback=5000)
params["criterion"] = c.get("training", "criterion", fallback="nll_loss")
params["lr_start"] = c.getfloat("training", "lr_start", fallback=1e-4)
params["ignore_frames_loss"] = c.getint("training", "ignore_frames_loss")
if (params["criterion"] == "focal_loss") or (params["criterion"] == "sum_losses"):
    params["gamma"] = c.getfloat("training", "gamma", fallback=2.0)
if params["criterion"] == "sum_losses":
    params["w"] = c.getfloat("training", "w", fallback=0.5)
params["cuda"] = c.getboolean("training", "cuda")
params["scheduler"] = c.get("training", "scheduler", fallback=None)
if params["scheduler"] == "step":
    params["scheduler_step_size"] = c.getint("training", "step_size")
    params["scheduler_gamma"] = c.getfloat("training", "gamma")

# dataset params
params["relative_path"] = c.get("dataset", "relative_path")
params["dataset_size"] = c.get("dataset", "dataset_size", fallback="full")
params["batch_size"] = c.getint("dataset", "batch_size", fallback=1)
params["num_workers"] = 0#c.getint("dataset", "num_workers", fallback=1)
params["data_duration"] = c.getint("dataset", "data_duration")
params["data_step"] = c.getint("dataset", "data_step")
params["testing_data_step"] = c.getint("testing", "data_step")
params["data_smoothing"] = c.get("dataset", "data_smoothing", fallback="2d")
params["norm_video"] = c.get("dataset", "norm_video", fallback="chunk")
params["remove_background"] = c.get(
    "dataset", "remove_background", fallback="average"
)
params["only_sparks"] = c.getboolean("dataset", "only_sparks", fallback=False)
params["noise_data_augmentation"] = c.getboolean(
    "dataset", "noise_data_augmentation", fallback=False
)
params["sparks_type"] = c.get("dataset", "sparks_type", fallback="peaks")
params["inference"] = c.get("dataset", "inference", fallback="overlap")

# UNet params
params["nn_architecture"] = c.get(
    "network", "nn_architecture", fallback="pablos_unet"
)
params["unet_steps"] = c.getint("network", "unet_steps")
params["first_layer_channels"] = c.getint("network", "first_layer_channels")
params["num_channels"] = c.getint("network", "num_channels", fallback=1)
params["dilation"] = c.getboolean("network", "dilation", fallback=1)
params["border_mode"] = c.get("network", "border_mode")
params["batch_normalization"] = c.get(
    "network", "batch_normalization", fallback="none"
)
params["temporal_reduction"] = c.getboolean(
    "network", "temporal_reduction", fallback=False
)
params["initialize_weights"] = c.getboolean(
    "network", "initialize_weights", fallback=False
)
if params["nn_architecture"] == "github_unet":
    params["attention"] = c.getboolean("network", "attention")
    params["up_mode"] = c.get("network", "up_mode")

In [38]:
############################# configure wandb ##############################

wandb_log = c.getboolean("general", "wandb_enable", fallback=False)
if wandb_log:
    resume = "must" if params["load_epoch"] > 0 else None

    wandb.init(
        project=wandb_project_name,
        #name=params["run_name"],
        notes=c.get("general", "wandb_notes", fallback=None),
        id=params["run_name"],
        resume=resume
    )
    logging.getLogger("wandb").setLevel(logging.DEBUG)
    # wandb.save(CONFIG_FILE)

In [39]:
############################# print parameters #############################

logger.info("Command parameters:")
for k, v in params.items():
    logger.info(f"{k:>18s}: {v}")
    # load parameters to wandb
    if wandb_log:
        wandb.config[k] = v
    # TODO: AGGIUNGERE TUTTI I PARAMS NECESSARI DA PRINTARE

[10:41:34] [  INFO  ] [  __main__  ] < 3  > -- Command parameters:
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --           run_name: TEST6
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --      load_run_name: None
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --         load_epoch: 0
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --       train_epochs: 10
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --          criterion: lovasz_softmax
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --           lr_start: 0.001
[10:41:34] [  INFO  ] [  __main__  ] < 5  > -- ignore_frames_loss: 6
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --               cuda: True
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --          scheduler: step
[10:41:34] [  INFO  ] [  __main__  ] < 5  > -- scheduler_step_size: 10
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --    scheduler_gamma: 0.1
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --      relative_path: ../data/sparks_dataset
[10:41:34] [  INFO  ] [  __main__  ] < 5  > --   

In [40]:
############################ configure datasets ############################

# select samples that are used for training and testing
if params["dataset_size"] == "full":
    train_sample_ids = ["01","02","03","04","06","07","08","09",
                        "11","12","13","14","16","17","18","19",
                        "21","22","23","24","27","28","29","30",
                        "33","35","36","38","39","41","42","43",
                        "44","46"]
    test_sample_ids = ["05", "10", "15", "20", "25", "32", "34", "40", "45"]
elif params["dataset_size"] == "minimal":
    train_sample_ids = ["01"]
    test_sample_ids = ["34"]
else:
    logger.error(f"{params['dataset_size']} is not a valid dataset size.")
    exit()

# detect CUDA devices
if params["cuda"]:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pin_memory = True
else:
    device = "cpu"
    pin_memory = False
n_gpus = torch.cuda.device_count()
logger.info(f"Using torch device {device}, with {n_gpus} GPUs")

# set if temporal reduction is used
if params["temporal_reduction"]:
    logger.info(f"Using temporal reduction with {params['num_channels']} channels")

# normalize whole videos or chunks individually
if params["norm_video"] == "chunk":
    logger.info("Normalizing each chunk using min and max")
elif params["norm_video"] == "movie":
    logger.info("Normalizing whole video using min and max")
elif params["norm_video"] == "abs_max":
    logger.info("Normalizing whole video using 16-bit absolute max")

# initialize training dataset
dataset_path = os.path.realpath(f"{params['relative_path']}")
assert os.path.isdir(dataset_path), f'"{dataset_path}" is not a directory'
logger.info(f"Using {dataset_path} as dataset root path")
dataset = SparkDataset(
    base_path=dataset_path,
    sample_ids=train_sample_ids,
    testing=False,
    smoothing=params["data_smoothing"],
    step=params["data_step"],
    duration=params["data_duration"],
    remove_background=params["remove_background"],
    temporal_reduction=params["temporal_reduction"],
    num_channels=params["num_channels"],
    normalize_video=params["norm_video"],
    only_sparks=params["only_sparks"],
    sparks_type=params["sparks_type"],
    ignore_index=ignore_index,
    inference=None,
)

# transforms are applied when getting a sample from the dataset
if params["noise_data_augmentation"]:
    dataset = unet.TransformedDataset(dataset, random_flip_noise)
else:
    dataset = unet.TransformedDataset(dataset, random_flip)

logger.info(f"Samples in training dataset: {len(dataset)}")

# initialize testing dataset
pattern_test_filenames = os.path.join(
    f"{dataset_path}", "videos_test", "[0-9][0-9]_video.tif"
)

testing_datasets = [
    SparkDataset(
        base_path=dataset_path,
        sample_ids=[sample_id],
        testing=True,
        smoothing=params["data_smoothing"],
        step=params["testing_data_step"],
        duration=params["data_duration"],
        remove_background=params["remove_background"],
        temporal_reduction=params["temporal_reduction"],
        num_channels=params["num_channels"],
        normalize_video=params["norm_video"],
        only_sparks=params["only_sparks"],
        sparks_type=params["sparks_type"],
        ignore_frames=params["ignore_frames_loss"],
        ignore_index=ignore_index,
        inference=params["inference"],
    )
    for sample_id in test_sample_ids
]

for i, tds in enumerate(testing_datasets):
    logger.info(f"Testing dataset {i} contains {len(tds)} samples")

# initialize data loaders
dataset_loader = DataLoader(
    dataset,
    batch_size=params["batch_size"],
    shuffle=True,
    num_workers=params["num_workers"],
    pin_memory=pin_memory,
)

[10:41:34] [  INFO  ] [  __main__  ] < 26 > -- Using torch device cuda, with 1 GPUs
[10:41:34] [  INFO  ] [  __main__  ] < 38 > -- Normalizing whole video using 16-bit absolute max
[10:41:34] [  INFO  ] [  __main__  ] < 43 > -- Using C:\Users\dotti\sparks_project\data\sparks_dataset as dataset root path
[10:41:34] [  INFO  ] [  __main__  ] < 67 > -- Samples in training dataset: 245
[10:41:34] [WARNING ] [tifffile.tifffile] <16549> -- TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
[10:41:34] [WARNING ] [tifffile.tifffile] <16549> -- TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
[10:41:34] [ DEBUG  ] [  datasets  ] <188 > -- Computing spark peaks...
[10:41:38] [ DEBUG  ] [  datasets  ] <195 > -- Sample 34 contains 16 sparks.
[10:41:38] [ DEBUG  ] [  datasets  ] <292 > -- Added padding of 24 frames to video with unsuitable duration
[10:41:38] [  INFO  ] [  __m

In [41]:
############################## configure UNet ##############################

if params["nn_architecture"] == "pablos_unet":

    batch_norm = {"batch": True, "none": False}

    unet_config = unet.UNetConfig(
        steps=params["unet_steps"],
        first_layer_channels=params["first_layer_channels"],
        num_classes=num_classes,
        ndims=ndims,
        dilation=params["dilation"],
        border_mode=params["border_mode"],
        batch_normalization=batch_norm[params["batch_normalization"]],
        num_input_channels=params["num_channels"],
    )

    if not params["temporal_reduction"]:
        network = unet.UNetClassifier(unet_config)
    else:
        assert (
            params["data_duration"] % params["num_channels"] == 0
        ), "using temporal reduction chunks_duration must be a multiple of num_channels"
        network = TempRedUNet(unet_config)

elif params["nn_architecture"] == "github_unet":
    network = UNet(
        in_channels=params["num_channels"],
        out_channels=num_classes,
        n_blocks=params["unet_steps"] + 1,
        start_filts=params["first_layer_channels"],
        up_mode=params["up_mode"],
        # up_mode = 'transpose', # TESTARE DIVERSE POSSIBILTÀ
        # up_mode='resizeconv_nearest',  # Enable to avoid checkerboard artifacts
        merge_mode="concat",  # Default, dicono che funziona meglio
        # planar_blocks=(0,), # magari capire cos'è e testarlo ??
        activation="relu",
        normalization=params[
            "batch_normalization"
        ],  # Penso che nell'implementazione di Pablo è 'none'
        attention=params["attention"],  # magari da testare con 'True' ??
        # full_norm=False,  # Uncomment to restore old sparse normalization scheme
        dim=ndims,
        conv_mode=params["border_mode"],  # 'valid' ha dei vantaggi a quanto pare...
    )

if device != "cpu":
    network = nn.DataParallel(network).to(device)
    torch.backends.cudnn.benchmark = True

if wandb_log:
    wandb.watch(network)

if params["initialize_weights"]:
    logger.info("Initializing UNet weights...")
    network.apply(weights_init)

In [42]:
########################### initialize training ############################

optimizer = optim.Adam(network.parameters(), lr=params["lr_start"])

if params["scheduler"] == "step":
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=params["scheduler_step_size"],
        gamma=params["scheduler_gamma"],
    )
else:
    scheduler = None

network.train()

output_path = os.path.join(output_relative_path, params["run_name"])
logger.info(f"Output directory: {output_path}")

summary_writer = SummaryWriter(os.path.join(output_path, "summary"), purge_step=0)

if params["load_run_name"] != None:
    load_path = os.path.join(output_relative_path, params["load_run_name"])
    logger.info(f"Model loaded from directory: {load_path}")
else:
    load_path = None

# class weights
if params["criterion"] in ["nll_loss", "focal_loss", "sum_losses"]:
    class_weights = compute_class_weights(dataset)
    logger.info(
        "Using class weights: {}".format(
            ", ".join(str(w.item()) for w in class_weights)
        )
    )

if params["criterion"] == "nll_loss":
    criterion = nn.NLLLoss(
        ignore_index=ignore_index, weight=class_weights.to(device)
    )
elif params["criterion"] == "focal_loss":
    criterion = FocalLoss(
        reduction="mean",
        ignore_index=ignore_index,
        alpha=class_weights,
        gamma=params["gamma"],
    )
elif params["criterion"] == "lovasz_softmax":
    criterion = LovaszSoftmax3d(
        classes="present", per_image=False, ignore=ignore_index
    )
elif params["criterion"] == "sum_losses":
    criterion = SumFocalLovasz(
        classes="present",
        per_image=False,
        ignore=ignore_index,
        alpha=class_weights,
        gamma=params["gamma"],
        reduction="mean",
        w=params["w"],
    )

# directory where predicted class movies are saved
preds_output_dir = os.path.join(output_path, "predictions")
os.makedirs(preds_output_dir, exist_ok=True)

# generate dict of managed objects
managed_objects = {"network": network, "optimizer": optimizer}
if scheduler is not None:
    managed_objects["scheduler"] = scheduler

trainer = myTrainingManager(
    # training items
    training_step=lambda _: training_step(
        sampler=sampler,
        network=network,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        criterion=criterion,
        dataset_loader=dataset_loader,
        ignore_frames=params["ignore_frames_loss"],
    ),
    save_every=c.getint("training", "save_every", fallback=5000),
    # load_path=load_path,
    save_path=output_path,
    managed_objects=unet.managed_objects(managed_objects),
    # testing items
    test_function=lambda _: test_function(
        network=network,
        device=device,
        criterion=criterion,
        testing_datasets=testing_datasets,
        ignore_frames=params["ignore_frames_loss"],
        training_name=params["run_name"],
        output_dir=preds_output_dir,
        training_mode=True,
        debug=debug_mode,
    ),
    test_every=c.getint("training", "test_every", fallback=1000),
    plot_every=c.getint("training", "test_every", fallback=1000),
    summary_writer=summary_writer,
)

[10:41:38] [  INFO  ] [  __main__  ] < 17 > -- Output directory: runs\TEST6


In [43]:
############################ init random seeds #############################
import random

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [44]:
############################## start training ##############################

if params["load_epoch"] != 0:
    trainer.load(params["load_epoch"])

if c.getboolean("general", "training", fallback=False):  # Run training procedure on data
    # logger.info("Validate network before training")
    # trainer.run_validation(wandb_log=wandb_log)
    logger.info("Starting training")
    trainer.train(
        params["train_epochs"],
        print_every=c.getint("training", "print_every", fallback=100),
        wandb_log=wandb_log,
    )


[10:41:38] [  INFO  ] [  __main__  ] < 9  > -- Starting training
[10:41:40] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.001
[10:41:41] [  INFO  ] [training_inference_tools] <102 > -- Iteration 0...
[10:41:41] [  INFO  ] [training_inference_tools] <103 > -- 	Training loss: 2.474
[10:41:41] [  INFO  ] [training_inference_tools] <104 > -- 	Time elapsed: 3.42s
[10:41:42] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.001
[10:41:44] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.001
[10:41:47] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.001
[10:41:49] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.001
[10:41:51] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.001
[10:41:55] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.001
[10:41:58] [ DEBUG  ] [training_inference_tools] <199 > -- Current learning rate: 0.

In [ ]:
if c.getboolean("general", "testing", fallback=False):  # Run final validation
    logger.info("Starting final validation")
    trainer.run_validation(wandb_log=wandb_log)

[14:29:19] [  INFO  ] [  __main__  ] < 2  > -- Starting final validation
[14:29:19] [  INFO  ] [training_inference_tools] < 45 > -- Validating network at iteration 40...
[14:29:19] [ DEBUG  ] [training_inference_tools] <525 > -- Testing function: running sample 34 in UNet
[14:29:34] [ DEBUG  ] [in_out_tools] <278 > -- Writing videos on directory c:\Users\dotti\sparks_project\sparks\runs\TEST5\predictions ..
[14:29:35] [ DEBUG  ] [training_inference_tools] <555 > -- Time to run sample 34 in UNet: 15.32 s
[14:29:35] [ DEBUG  ] [training_inference_tools] <562 > -- Testing function: re-organising annotations
[14:29:39] [ DEBUG  ] [training_inference_tools] <591 > -- Time to re-organise annotations: 3.89 s
[14:29:39] [ DEBUG  ] [training_inference_tools] <597 > -- Testing function: getting processed output (segmentation and instances)
[14:29:39] [ DEBUG  ] [data_processing_tools] <448 > -- Events detection threshold: 0.804
[14:29:48] [ DEBUG  ] [data_processing_tools] <514 > -- Number of sp

KeyboardInterrupt: 

In [ ]:
if wandb_log:
    wandb.finish()


# Visualize UNet architecture

In [ ]:
# get number of trainable parameters
num_params = sum(p.numel() for p in network.parameters() if p.requires_grad)
logger.debug(f"Number of trainable parameters: {num_params}")

In [ ]:
# get dummy unet input
batch = next(iter(dataset_loader))
x = batch[0].to(device)
yhat = network(x[:,None]) # Give dummy batch to forward()

In [ ]:
from torchviz import make_dot

In [ ]:
make_dot(yhat, params=dict(list(network.named_parameters()))).render("unet_model", format="png")